# Recommender System - Collaborative Filtering

Collaborative filtering is a type of recommender system. It groups users based on similar behavior, recommending new items according to group characteristics.

Collaborative filtering is an information retrieval method that recommends items to users based on how other users with similar preferences and behavior have interacted with that item. In other words, **collaborative filtering algorithms group users based on behavior and use general group characteristics to recommend items to a target user**. Collaborative recommender systems operate on the principle that similar users (behavior-wise) share similar interests and similar tastes.

**Collaborative filtering vs content-based filtering**

Collaborative filtering is one of two primary types of recommender systems, the other being content-based recommenders. This latter method uses item features to recommend similar items as the items with which a particular user has positively interacted in the past. 

While collaborative filtering focuses on user similarity to recommend items, content-based filtering recommends items exclusively according to item profile features. Content-based filtering targets recommendations to one specific user’s preferences rather than a group or type as in collaborative filtering.
Together, collaborative and content-based systems form **hybrid recommender systems**, that is be implemented latter on.

In [22]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## User-item matrix

Has in a standard setting of collaborative filtering, here we have a set of n users and a set of x items (business/restaurants). Each user’s individual preference for each item is displayed in a user-item matrix (sometimes called a user rating matrix). 

Here, users are represented in rows and items in columns. In the Rij matrix, a given value represents the behavior of user u toward item i.

In [27]:
Philadelphia_reviews = pd.read_csv('/Users/mac/Downloads/ADC-MECD/data/Philadelphia_reviews.csv')
user_dataframe = pd.read_csv('/Users/mac/Downloads/ADC-MECD/data/user_dataframe.csv')

In [25]:
#Philadelphia_reviews = Philadelphia_reviews[Philadelphia_reviews['year'] == 2012]

In [28]:
num_unique_users = Philadelphia_reviews['user_id'].nunique()
print(f"Number of unique users: {num_unique_users}")

num_unique_restaurants = Philadelphia_reviews['business_id'].nunique()
print(f"Number of unique restaurants: {num_unique_restaurants}")

num_unique_reviews = Philadelphia_reviews['review_id'].nunique()
print(f"Number of unique reviews: {num_unique_reviews}")

Number of unique users: 5916
Number of unique restaurants: 501
Number of unique reviews: 10407


In [29]:
Philadelphia_review = Philadelphia_reviews[['user_id', 'stars', 'business_id', 'restaurant_name']]
Philadelphia_review

user_id  stars             business_id   restaurant_name
0      3fTw9hmzlbMK2Q32Q-a4Hw      4  kxX2SOes4o-D3ZQBkiMRfA             Zaika
1      c4rYJcGZ0gRl-C6SqbehQQ      3  kxX2SOes4o-D3ZQBkiMRfA             Zaika
2      EMMKdK1cGrLSo08wgSYHJQ      5  kxX2SOes4o-D3ZQBkiMRfA             Zaika
3      fbV1aq9ASXggQyGTNN-hzg      5  kxX2SOes4o-D3ZQBkiMRfA             Zaika
4      TeLcAcuphfDfg-lgehu2qg      5  kxX2SOes4o-D3ZQBkiMRfA             Zaika
...                       ...    ...                     ...               ...
10402  2SYFAGATQn96D8-FZNvaMw      4  7pwZZVVlYCxQvVdd8Q03wg  La Rosa Pizzeria
10403  HwL0cUMC7-qghiAKBzMpXg      5  7pwZZVVlYCxQvVdd8Q03wg  La Rosa Pizzeria
10404  UQIERttr5tWwid4TncXLvw      4  7pwZZVVlYCxQvVdd8Q03wg  La Rosa Pizzeria
10405  ygvP779WApkVWlHiamB_Uw      5  7pwZZVVlYCxQvVdd8Q03wg  La Rosa Pizzeria
10406  yIsQ4qFCyY-clmlRwQlw_w      5  7pwZZVVlYCxQvVdd8Q03wg  La Rosa Pizzeria

[10407 rows x 4 columns]

In [30]:
user = pd.read_csv('/Users/mac/Downloads/ADC-MECD/data/user_dataframe.csv')
user=user.drop_duplicates()

In [31]:
user_item_matrix = Philadelphia_review.pivot_table(
    index='user_id',           # Rows (Users)
    columns='restaurant_name', # Columns (Restaurants)
    values='stars',             # Values (Ratings)
    fill_value=np.nan          # Keep NaN for missing values
)

In [32]:
user_item_matrix

restaurant_name         17 Snyder Chinese  4 Seasons Food Market  \
user_id                                                            
--pMssoWlOqX0pLiPMiZZw                NaN                    NaN   
-00kdEIhCt-ODaV4BS-EAg                NaN                    NaN   
-0aZWYi2YicFaLxTru96nA                NaN                    NaN   
-2Pn4YRNRgJM8foLYkkv7g                NaN                    NaN   
-2_eYCiY4JXD_hUI2_HK6w                NaN                    NaN   
...                                   ...                    ...   
zvq6qxc2IVA08jaUelUMRw                NaN                    NaN   
zwCVNO9Kkv6-XqRnNliKgQ                NaN                    NaN   
zwCkk1Pd88uZpvIQRRFo-A                NaN                    NaN   
zx43prq7t59Bg5xDOm1t3g                NaN                    NaN   
zziJLt25YU6dp01sewR-IQ                NaN                    NaN   

restaurant_name         A P J Texas Weiner  Abyssinia  \
user_id                                                 
--pMssoWlOqX0pLiPMiZZw                 NaN        NaN   
-00kdEIhCt-ODaV4BS-EAg                 NaN        NaN   
-0aZWYi2YicFaLxTru96nA                 NaN        NaN   
-2Pn4YRNRgJM8foLYkkv7g                 NaN        NaN   
-2_eYCiY4JXD_hUI2_HK6w                 NaN        NaN   
...                                    ...        ...   
zvq6qxc2IVA08jaUelUMRw                 NaN        NaN   
zwCVNO9Kkv6-XqRnNliKgQ                 NaN        NaN   
zwCkk1Pd88uZpvIQRRFo-A                 NaN        NaN   
zx43prq7t59Bg5xDOm1t3g                 NaN        NaN   
zziJLt25YU6dp01sewR-IQ                 NaN        NaN   

restaurant_name         Al's Italiano Pizzeria & Ristorante  Allegro Pizza  \
user_id                                                                      
--pMssoWlOqX0pLiPMiZZw                                  NaN            NaN   
-00kdEIhCt-ODaV4BS-EAg                                  NaN            NaN   
-0aZWYi2YicFaLxTru96nA                                  NaN            NaN   
-2Pn4YRNRgJM8foLYkkv7g                                  NaN            NaN   
-2_eYCiY4JXD_hUI2_HK6w                                  NaN            NaN   
...                                                     ...            ...   
zvq6qxc2IVA08jaUelUMRw                                  NaN            NaN   
zwCVNO9Kkv6-XqRnNliKgQ                                  NaN            NaN   
zwCkk1Pd88uZpvIQRRFo-A                                  NaN            NaN   
zx43prq7t59Bg5xDOm1t3g                                  NaN            NaN   
zziJLt25YU6dp01sewR-IQ                                  NaN            NaN   

restaurant_name         Alyan's Restaurant  Amada  Amuse  Angela's Pizza  ...  \
user_id                                                                   ...   
--pMssoWlOqX0pLiPMiZZw                 NaN    NaN    NaN             NaN  ...   
-00kdEIhCt-ODaV4BS-EAg                 NaN    NaN    NaN             NaN  ...   
-0aZWYi2YicFaLxTru96nA                 NaN    NaN    NaN             NaN  ...   
-2Pn4YRNRgJM8foLYkkv7g                 NaN    NaN    NaN             NaN  ...   
-2_eYCiY4JXD_hUI2_HK6w                 NaN    NaN    NaN             NaN  ...   
...                                    ...    ...    ...             ...  ...   
zvq6qxc2IVA08jaUelUMRw                 5.0    NaN    NaN             NaN  ...   
zwCVNO9Kkv6-XqRnNliKgQ                 NaN    NaN    NaN             NaN  ...   
zwCkk1Pd88uZpvIQRRFo-A                 NaN    NaN    NaN             NaN  ...   
zx43prq7t59Bg5xDOm1t3g                 NaN    NaN    NaN             NaN  ...   
zziJLt25YU6dp01sewR-IQ                 NaN    NaN    NaN             NaN  ...   

restaurant_name         Winnie's Manayunk  Wokano  Wolf Street Pizza & Grill  \
user_id                                                                        
--pMssoWlOqX0pLiPMiZZw                NaN     NaN                        NaN   
-00kdEIhCt-ODaV4BS-EAg                NaN     NaN               

In [35]:
user_item_matrix.to_csv('/Users/mac/Downloads/ADC-MECD/data/user_item_matrix.csv')

## Similarity measures

Proximity in vector space is a primary method to determine similarity between various users collaborative recommendation algorithm. But the specific metrics used to determine that proximity may vary. Two such metrics are cosine similarity and Pearson correlation coefficient.


**Cosine similarity**

Cosine similarity signifies the measurement of the angle between two vectors. Compared vectors comprise a subset of ratings for given user or item. The cosine similarity score can be any value between -1 and 1. The higher the cosine score, the more alike two items are considered. Some sources recommend this metric for 
high-dimensional feature spaces. In collaborative filtering, vector points are pulled directly from the user-item matrix. Cosine similarity is represented by this formula, where x and y signify two vectors in vector space.


**Pearson correlation coefficient (PCC)**

PCC helps measure similarity between items or users by computing the correlation between two users’ or items’ respective ratings. PCC ranges between -1 and 1, which signify negative to identical correlation. Unlike cosine similarity, PCC uses all the ratings for a given user or item. For example, if calculating PCC between two users, we use this formula, in which a and b are different users, and rai and rbi are that user's rating for item i.

## Memory-based Recommender Systems

There are two primary types of collaborative filtering systems: memory-based and model-based.


Memory-based recommender systems, or neighbor-based systems, are extensions of k-nearest neighbors classifiers because they attempt to predict a target user’s behavior toward a given item based on similar users or set of items. 

### User-Based Collaborative Filtering

**User-based filtering** recommends items to a target user based on the preferences of behaving users. The recommendation algorithm compares a target user’s past behavior to other users. Specifically, the system assigns each user a weight representing their perceived similarity with the target user—this is the target user’s neighbors. 


It then selects n users with the highest weights and computes a prediction of the target user’s behavior from a weighted average of the selected neighbors’ behavior. The system then recommends items to the target user based on this prediction. The principle is that, if the target user behaved similarly to this group in the past, they will behave similarly with unseen items. User-based similarity functions are computed between rows in the user-item matrix.


In [11]:
# Normalize the ratings by subtracting each user's average
user_ratings_mean = user_item_matrix.mean(axis=1)
normalized_matrix = user_item_matrix.sub(user_ratings_mean, axis=0)

# Fill NaN with 0s for similarity computation
normalized_matrix.fillna(0, inplace=True)

# Compute cosine similarity between users
user_similarity = cosine_similarity(normalized_matrix)

In [12]:
# Predict ratings
def predict_ratings(user_id, item_id):
    # Get similar users
    user_index = normalized_matrix.index.get_loc(user_id)
    similarities = user_similarity[user_index]
    
    # Get other users who rated the item
    item_ratings = user_item_matrix[item_id]
    non_zero_ratings = item_ratings[item_ratings.notnull()]

    # Calculate weighted sum of ratings
    weighted_sum = 0
    sim_sum = 0
    for other_user_id in non_zero_ratings.index:
        other_user_index = normalized_matrix.index.get_loc(other_user_id)
        sim = similarities[other_user_index]
        if sim != 0:
            rating_diff = non_zero_ratings[other_user_id] - user_ratings_mean[other_user_id]
            weighted_sum += sim * rating_diff
            sim_sum += abs(sim)
    
    # Final predicted rating
    if sim_sum != 0:
        predicted_rating = user_ratings_mean[user_id] + (weighted_sum / sim_sum)
    else:
        predicted_rating = user_ratings_mean[user_id]
    
    return predicted_rating

In [13]:
# Example usage
predicted_rating = predict_ratings('zvq6qxc2IVA08jaUelUMRw', '4 Seasons Food Market')
print(predicted_rating)

3.8333333333333335


In [14]:
# Example usage
predicted_rating = predict_ratings('zvq6qxc2IVA08jaUelUMRw', 'Allegro Pizza')
print(predicted_rating)

2.5225225225225225


In [15]:
# Function to get top 10 restaurant recommendations for a specific user
def recommend_restaurants(user_id, min_rating=3, top_n=10):
    recommendations = []
    
    # Loop through all restaurants
    for restaurant_id in user_item_matrix.columns:
        # Only predict rating for restaurants the user hasn't rated yet
        if pd.isna(user_item_matrix.loc[user_id, restaurant_id]):
            predicted_rating = predict_ratings(user_id, restaurant_id)
            
            # Only recommend restaurants with predicted ratings >= min_rating
            if predicted_rating >= min_rating:
                recommendations.append((restaurant_id, predicted_rating))
    
    # Sort recommendations by predicted rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    # Return only the top N recommendations
    return recommendations[:top_n]


In [16]:
# Example usage for a specific user
user_id = 'zvq6qxc2IVA08jaUelUMRw'
top_10_recommended_restaurants = recommend_restaurants(user_id)

# Display the top 10 recommendations
print(f"Top 10 Recommendations for user {user_id}:")
for restaurant, predicted_rating in top_10_recommended_restaurants:
    print(f"Restaurant: {restaurant}, Predicted Rating: {predicted_rating:.2f}")

Top 10 Recommendations for user zvq6qxc2IVA08jaUelUMRw:
Restaurant: Axis Pizza, Predicted Rating: 6.13
Restaurant: Buffalo Wild Wings, Predicted Rating: 6.03
Restaurant: Tir na nOg Irish Pub, Predicted Rating: 6.03
Restaurant: Cavanaugh's Restaurant, Predicted Rating: 5.79
Restaurant: Octopus Falafel Truck, Predicted Rating: 5.52
Restaurant: Red Lobster, Predicted Rating: 5.43
Restaurant: Xochitl, Predicted Rating: 5.31
Restaurant: Frank's Alot, Predicted Rating: 5.13
Restaurant: Popi's Italian Restaurant, Predicted Rating: 5.06
Restaurant: The Twisted Tail, Predicted Rating: 5.04


The predicted rating of 5.80 is above the typical rating range, as ratings usually range between 1 and 5.
This is likely due to the normalization step, where the predicted rating is computed based on the average user rating and the weighted sum of other users' ratings.

To prevent predicted ratings from exceeding the standard range, we will cap the predicted ratings at 5 and set a lower bound of 1.
This will ensure the recommendations fall within the expected range.

In [17]:
# Function to predict the rating for a specific user and restaurant
def predict_rating(user_id, item_id):
    # Get similar users
    user_index = normalized_matrix.index.get_loc(user_id)
    similarities = user_similarity[user_index]
    
    # Get other users who rated the item
    item_ratings = user_item_matrix[item_id]
    non_zero_ratings = item_ratings[item_ratings.notnull()]

    # Calculate weighted sum of ratings
    weighted_sum = 0
    sim_sum = 0
    for other_user_id in non_zero_ratings.index:
        other_user_index = normalized_matrix.index.get_loc(other_user_id)
        sim = similarities[other_user_index]
        if sim != 0:
            rating_diff = non_zero_ratings[other_user_id] - user_ratings_mean[other_user_id]
            weighted_sum += sim * rating_diff
            sim_sum += abs(sim)
    
    # Final predicted rating
    if sim_sum != 0:
        predicted_rating = user_ratings_mean[user_id] + (weighted_sum / sim_sum)
    else:
        predicted_rating = user_ratings_mean[user_id]

    # Cap the predicted rating between 1 and 5
    predicted_rating = min(max(predicted_rating, 1), 5)
    
    return predicted_rating

# Function to get top 10 restaurant recommendations for a specific user
def recommend_restaurants(user_id, min_rating=3, top_n=10):
    recommendations = []
    
    # Loop through all restaurants
    for restaurant_id in user_item_matrix.columns:
        # Only predict rating for restaurants the user hasn't rated yet
        if pd.isna(user_item_matrix.loc[user_id, restaurant_id]):
            predicted_rating = predict_rating(user_id, restaurant_id)
            
            # Only recommend restaurants with predicted ratings >= min_rating
            if predicted_rating >= min_rating:
                recommendations.append((restaurant_id, predicted_rating))
    
    # Sort recommendations by predicted rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    # Return only the top N recommendations
    return recommendations[:top_n]

In [18]:
# Example usage for a specific user
user_id = 'zvq6qxc2IVA08jaUelUMRw'
top_10_recommended_restaurants = recommend_restaurants(user_id,min_rating=3, top_n=15)

# Display the top 10 recommendations
print(f"Top 10 Recommendations for user {user_id}:")
for restaurant, predicted_rating in top_10_recommended_restaurants:
    print(f"Restaurant: {restaurant}, Predicted Rating: {predicted_rating:.2f}")

Top 10 Recommendations for user zvq6qxc2IVA08jaUelUMRw:
Restaurant: Axis Pizza, Predicted Rating: 5.00
Restaurant: Buffalo Wild Wings, Predicted Rating: 5.00
Restaurant: Cavanaugh's Restaurant, Predicted Rating: 5.00
Restaurant: Con Murphy's, Predicted Rating: 5.00
Restaurant: Five Guys, Predicted Rating: 5.00
Restaurant: Frank's Alot, Predicted Rating: 5.00
Restaurant: Octopus Falafel Truck, Predicted Rating: 5.00
Restaurant: Popi's Italian Restaurant, Predicted Rating: 5.00
Restaurant: Red Lobster, Predicted Rating: 5.00
Restaurant: The Twisted Tail, Predicted Rating: 5.00
Restaurant: Tir na nOg Irish Pub, Predicted Rating: 5.00
Restaurant: Tokai Sushi Hibachi Steakhouses, Predicted Rating: 5.00
Restaurant: Xochitl, Predicted Rating: 5.00
Restaurant: Judah Mediterranean Grill, Predicted Rating: 4.90
Restaurant: Otolith Sustainable Seafood, Predicted Rating: 4.90


### Item-Based Collaborative Filtering

**Item-based filtering** recommends new items to a target user based on that user’s behavior toward similar items. Note, however, that in comparing items, the collaborative system does not compare item features (as in content-based filtering) but instead how users interact with those items. 

In [19]:
# Compute cosine similarity between items (restaurants)
item_similarity = cosine_similarity(normalized_matrix.T)  # Transpose to get item-item similarity
item_similarity = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
item_similarity

restaurant_name                      17 Snyder Chinese  4 Seasons Food Market  \
restaurant_name                                                                 
17 Snyder Chinese                                  1.0                    0.0   
4 Seasons Food Market                              0.0                    1.0   
A P J Texas Weiner                                 0.0                    0.0   
Abyssinia                                          0.0                    0.0   
Al's Italiano Pizzeria & Ristorante                0.0                    0.0   
...                                                ...                    ...   
Zaffron                                            0.0                    0.0   
Zahav                                              0.0                    0.0   
Zaika                                              0.0                    0.0   
Zio Pizza Palace & Grill                           0.0                    0.0   
Zorba's Taverna                                    0.0                    0.0   

restaurant_name                      A P J Texas Weiner  Abyssinia  \
restaurant_name                                                      
17 Snyder Chinese                                   0.0   0.000000   
4 Seasons Food Market                               0.0   0.000000   
A P J Texas Weiner                                  0.0   0.000000   
Abyssinia                                           0.0   1.000000   
Al's Italiano Pizzeria & Ristorante                 0.0   0.000000   
...                                                 ...        ...   
Zaffron                                             0.0   0.000000   
Zahav                                               0.0  -0.041584   
Zaika                                               0.0   0.035969   
Zio Pizza Palace & Grill                            0.0   0.000000   
Zorba's Taverna                                     0.0   0.000000   

restaurant_name                      Al's Italiano Pizzeria & Ristorante  \
restaurant_name                                                            
17 Snyder Chinese                                                    0.0   
4 Seasons Food Market                                                0.0   
A P J Texas Weiner                                                   0.0   
Abyssinia                                                            0.0   
Al's Italiano Pizzeria & Ristorante                                  1.0   
...                                                                  ...   
Zaffron                                                              0.0   
Zahav                                                                0.0   
Zaika                                                                0.0   
Zio Pizza Palace & Grill                                             0.0   
Zorba's Taverna                                                      0.0   

restaurant_name                      Allegro Pizza  Alyan's Restaurant  \
restaurant_name                                                          
17 Snyder Chinese                         0.000000            0.000000   
4 Seasons Food Market                     0.000000            0.000000   
A P J Texas Weiner                        0.000000            0.000000   
Abyssinia                                 0.195341            0.000000   
Al's Italiano Pizzeria & Ristorante       0.000000            0.000000   
...                                            ...                 ...   
Zaffron                                   0.000000            0.000000   
Zahav                                    -0.037963           -0.093373   
Zaika                                     0.000000            0.000000   
Zio Pizza Palace & Grill                  0.000000            0.000000   
Zorba's Taverna                           0.000000            0.000000   

restaurant_name                         Amada     Amuse  Angela's Pizza  ...  \
res

In [20]:
# Function to predict the rating for a specific user and restaurant using item-item similarity
def predict_item_rating(user_id, item_id):
    # Get all the items (restaurants) the user has rated
    user_ratings = user_item_matrix.loc[user_id]
    
    # Get the items (restaurants) that are similar to the target item
    similar_items = item_similarity[item_id]
    
    # Only consider the items that the user has rated
    rated_items = user_ratings[user_ratings.notnull()].index
    similar_items = similar_items[rated_items]
    
    # Calculate weighted sum of the user's ratings for similar items
    weighted_sum = 0
    sim_sum = 0
    for similar_item in similar_items.index:
        similarity = similar_items[similar_item]
        rating = user_ratings[similar_item]
        weighted_sum += similarity * rating
        sim_sum += abs(similarity)
    
    # Final predicted rating
    if sim_sum != 0:
        predicted_rating = weighted_sum / sim_sum
    else:
        predicted_rating = user_ratings_mean[user_id]  # Fallback to user's average if no similar items
    
    # Cap the predicted rating between 1 and 5
    predicted_rating = min(max(predicted_rating, 1), 5)
    
    return predicted_rating

# Function to get top 10 restaurant recommendations for a specific user using item-item collaborative filtering
def recommend_restaurants_item_based(user_id, min_rating=3, top_n=10):
    recommendations = []
    
    # Loop through all restaurants
    for restaurant_id in user_item_matrix.columns:
        # Only predict rating for restaurants the user hasn't rated yet
        if pd.isna(user_item_matrix.loc[user_id, restaurant_id]):
            predicted_rating = predict_item_rating(user_id, restaurant_id)
            
            # Only recommend restaurants with predicted ratings >= min_rating
            if predicted_rating >= min_rating:
                recommendations.append((restaurant_id, predicted_rating))
    
    # Sort recommendations by predicted rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    # Return only the top N recommendations
    return recommendations[:top_n]

In [21]:
# Example usage for a specific user
user_id = 'zvq6qxc2IVA08jaUelUMRw'
top_10_recommended_restaurants = recommend_restaurants_item_based(user_id, min_rating=3, top_n=15)

# Display the top 10 recommendations
print(f"Top 10 Item-Item Recommendations for user {user_id}:")
for restaurant, predicted_rating in top_10_recommended_restaurants:
    print(f"Restaurant: {restaurant}, Predicted Rating: {predicted_rating:.2f}")


Top 10 Item-Item Recommendations for user zvq6qxc2IVA08jaUelUMRw:
Restaurant: Bellini, Predicted Rating: 5.00
Restaurant: Café y Chocolate, Predicted Rating: 5.00
Restaurant: Crazy Sushi, Predicted Rating: 5.00
Restaurant: Currito, Predicted Rating: 5.00
Restaurant: Essene Market & Café, Predicted Rating: 5.00
Restaurant: Five Guys, Predicted Rating: 5.00
Restaurant: National Mechanics, Predicted Rating: 5.00
Restaurant: Popi's Italian Restaurant, Predicted Rating: 5.00
Restaurant: Wawa, Predicted Rating: 5.00
Restaurant: Wokano, Predicted Rating: 4.87
Restaurant: Last Drop, Predicted Rating: 4.74
Restaurant: Chapterhouse Café & Gallery, Predicted Rating: 4.41
Restaurant: Fu-Wah Mini Market, Predicted Rating: 4.30
Restaurant: Prohibition Taproom, Predicted Rating: 4.25
Restaurant: Seorabol Korean Restaurant, Predicted Rating: 4.22
